In [1]:
import pandas as pd
import configparser
import pymysql
from sqlalchemy import create_engine
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
import warnings; warnings.simplefilter('ignore')

In [2]:
parser = configparser.ConfigParser()
parser.read("config\mysql.conf")
hostname = parser.get("mysql_config", "hostname")
username = parser.get("mysql_config", "username")
password = parser.get("mysql_config", "password")
dbname = parser.get("mysql_config", "database")
port = parser.get("mysql_config", "port")

conn_str = 'mysql+pymysql://' + username + ':' + password + '@' + hostname + '/' + dbname
db_connection = create_engine(conn_str)

Spróbować isolation forest
from sklearn.ensemble import IsolationForest

iso = IsolationForest(contamination=0.1)
outliers = iso.fit_predict(scaled_data)
data_no_outliers = scaled_data[outliers != 
Spróbować DBSCAN-1]


Wyczytanie danych z MySQL

In [3]:
ratings = pd.read_sql('SELECT * FROM movies_dwh.v_ml_ratings', con = db_connection)
ratings.drop(['rating_date', 'rating_time'], axis=1, inplace=True)

Macierz ocen

In [4]:
ratings_matrix = ratings.pivot(index='user_id', columns='movie_id', values='rating')
ratings_matrix = ratings_matrix.fillna(0)
#X = ratings_matrix.values

In [5]:
def normalize_row(row):
    return row - row.mean()
normalized_ratings = ratings_matrix.apply(normalize_row, axis=1)
X = normalized_ratings.values

In [6]:
pearson = normalized_ratings.T.corr(method='pearson')

In [7]:
def get_top_5_second_to_sixth_with_labels(row):
    largest = row.nlargest(6).iloc[1:6]
    return pd.Series(largest.index, index=largest.values)

new_df = pearson.apply(get_top_5_second_to_sixth_with_labels, axis=1)
result_pearson = new_df.stack().reset_index()
result_pearson.columns = ['user_id', 'similarity', 'similar_users']
#result_pearson

In [8]:
def find_common_movies(similarity_df, ratings_df):
    common_movies = {}
    for user_id in similarity_df['user_id'].unique():
        similar_users = similarity_df[similarity_df['user_id'] == user_id]['similar_users'].unique()
        watched_movies = ratings_df[ratings_df['user_id'].isin(similar_users)]['movie_id']
        common_movies[user_id] = watched_movies.value_counts()[watched_movies.value_counts() > 1].index.tolist()
    return common_movies

common_movies = find_common_movies(result_pearson, ratings)
print("Common Movies:")
print(common_movies)


Common Movies:
{1: [1371, 256, 1339, 31, 4085, 1275, 1172, 1343, 2336, 914, 1405, 1129, 673, 585], 2: [590, 185, 480, 500, 410, 377, 367, 527, 356, 344, 539, 296, 454, 589, 208, 292, 457, 597, 39, 150, 47, 110, 165, 434, 21, 509, 380, 318, 364, 50, 350, 349, 595, 587, 317, 316, 588, 300, 153, 288, 253, 10, 592, 161, 515, 593, 474, 282, 2, 442, 329, 34, 32, 440, 160, 225, 266, 231, 339, 357, 420, 432, 435, 586, 608, 235, 348, 508, 337, 95, 265, 236, 19, 173, 553, 315, 105, 36, 25, 17], 3: [356, 318, 593, 1, 2959, 527, 480, 4306, 296, 7438, 110, 2028, 3147, 48516, 1721, 2762, 457, 1961, 3578, 1270, 1198, 6874, 2628, 58559, 733, 4993, 590, 5952, 7153, 6539, 150, 6377, 208, 231, 344, 2571, 4226, 3105, 5418, 6711, 47, 1097, 2706, 1240, 4963, 2804, 2858, 736, 4973, 4878, 2716, 33794, 7361, 1206, 1831, 49272, 8360, 5989, 30707, 8961, 2683, 3949, 3717, 44191, 592, 1580, 1089, 858, 780, 2329, 165, 364, 51662, 377, 1265, 380, 59315, 60069, 56367, 2324, 292, 1682, 1704, 260, 161, 46578, 48394, 63

In [9]:
def predict_ratings(similarity_df, ratings_df, common_movies):
    predictions = []

    for user_id in common_movies.keys():
        similar_users = similarity_df[similarity_df['user_id'] == user_id]['similar_users'].unique()
        user_ratings = ratings_df[ratings_df['user_id'].isin(similar_users)]

        for movie_id in common_movies[user_id]:
            similar_users_ratings = user_ratings[user_ratings['movie_id'] == movie_id]
            if not similar_users_ratings.empty:
                predicted_rating = similar_users_ratings['rating'].mean()
                predictions.append({'user_id': user_id, 'movie_id': movie_id, 'predicted_rating': predicted_rating})

    return pd.DataFrame(predictions)

predicted_ratings_df = predict_ratings(result_pearson, ratings, common_movies)
print("Predicted Ratings DataFrame:")
print(predicted_ratings_df.head(15))


Predicted Ratings DataFrame:
    user_id  movie_id  predicted_rating
0         1      1371          2.125000
1         1       256          2.666667
2         1      1339          3.333333
3         1        31          3.500000
4         1      4085          3.333333
5         1      1275          1.500000
6         1      1172          3.000000
7         1      1343          4.500000
8         1      2336          2.250000
9         1       914          1.000000
10        1      1405          2.750000
11        1      1129          3.000000
12        1       673          1.000000
13        1       585          2.500000
14        2       590          3.600000


In [16]:
ratings_to_insert = predicted_ratings_df.values.tolist()

In [10]:
conn = pymysql.connect(
    host=hostname,
    user=username,
    password=password,
    db=dbname,
    port=int(port)
)

if conn is None:
    print("Error. Connection to MySQL cannot be established.")
else:
    print("Successfully connected to MySQL.")

cur = conn.cursor()

Successfully connected to MySQL.


In [13]:
create_table = """
CREATE TABLE IF NOT EXISTS movies_dwh.ml_result_user_user (
    user_id INT,
    movie_id INT,
    predicted_rating FLOAT
);
"""

insert_query = """
INSERT INTO movies_dwh.ml_result_user_user (user_id, movie_id, predicted_rating)
VALUES (%s, %s, %s);
"""

In [14]:
cur.execute(create_table)

0

In [17]:
# Wstawienie danych do tabeli
cur.executemany(insert_query, ratings_to_insert)

# Zatwierdzenie zmian w bazie danych
conn.commit()

# Zamknięcie kursora i połączenia
cur.close()
conn.close()

Walidacja

In [90]:
val = ratings
pearson_val = pearson
def predict_ratings(user_id, movie_id):
    if movie_id not in ratings_matrix.columns or user_id not in ratings_matrix.index:
        return np.nan

    # Znajdź podobnych użytkowników
    similar_users = pearson_val[user_id].sort_values(ascending=False).index[1:6]

    # Wyznacz średnią ocenę od podobnych użytkowników
    similar_ratings = ratings_matrix.loc[similar_users, movie_id]
    similar_ratings = similar_ratings[similar_ratings > 0]  # Usuń brakujące oceny

    if similar_ratings.empty:
        return np.nan

    return similar_ratings.mean()

# Przewidywanie ocen dla testowych danych
val['predicted_rating'] = val.apply(lambda row: predict_ratings(row['user_id'], row['movie_id']), axis=1)

# Usuń wiersze z brakującymi przewidywaniami
val = val.dropna(subset=['predicted_rating'])
val

,user_id,movie_id,rating,predicted_rating
0,1,31,2.5,3.50
2,1,1061,3.0,3.50
3,1,1129,2.0,3.00
4,1,1172,4.0,3.00
6,1,1287,2.0,0.50
...,...,...,...,...
99992,671,5816,4.0,4.00
99994,671,5952,5.0,4.30
99995,671,5989,4.0,4.00
99996,671,5991,4.5,4.00


In [ ]:
predicted_ratings_df = val[['user_id', 'movie_id', 'predicted_rating']]
#predicted_ratings_df.to_csv('predicted_ratings.csv', index=False)


In [93]:
# Rzeczywiste i przewidywane oceny
y_true = val['rating']
y_pred = val['predicted_rating']

# Obliczanie RMSE
rmse = sqrt(mean_squared_error(y_true, y_pred))
print(f'RMSE: {rmse}')

# Obliczanie MAE
mae = mean_absolute_error(y_true, y_pred)
print(f'MAE: {mae}')


RMSE: 1.057754192515523
MAE: 0.8013428690459659


Podobieństwo kosinusowe

In [65]:
cosine_sim = cosine_similarity(normalized_ratings)
cosine_sim_df = pd.DataFrame(cosine_sim_columns, index=normalized_ratings.index, columns=normalized_ratings.index)
#cosine_sim_df

In [67]:
new_df = cosine_sim_df.apply(get_top_5_second_to_sixth_with_labels, axis=1)
result_cosine = new_df.stack().reset_index()
result_cosine.columns = ['user_id', 'similarity', 'similar_users']
#result_cosine